<a href="https://colab.research.google.com/github/Codedeveloper123/Sentiment-Analysis-Model/blob/main/NoBE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline
 
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive 

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('gdrive/My Drive/NOBE/Nobe.csv', encoding = "latin-1")

In [ ]:
df.columns = ["label", "time", "date", "query", "username", "text"]

In [ ]:
df['label'][df['label']==4]=1

In [ ]:
df['label'][df['label']==2]=0

In [ ]:
df['label'][df['label']==0]=-1

In [ ]:
df

,label,time,date,query,username,text
0,-1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,-1,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,-1,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,-1,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,-1,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['text'] = df['text'].str.lower()
df['text']

0          is upset that he can't update his facebook by ...
1          @kenichan i dived many times for the ball. man...
2            my whole body feels itchy and like its on fire 
3          @nationwideclass no, it's not behaving at all....
4                              @kwesidei not the whole crew 
                                 ...                        
1599994    just woke up. having no school is the best fee...
1599995    thewdb.com - very cool to hear old walt interv...
1599996    are you ready for your mojo makeover? ask me f...
1599997    happy 38th birthday to my boo of alll time!!! ...
1599998    happy #charitytuesday @thenspcc @sparkscharity...
Name: text, Length: 1599999, dtype: object

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
 def clean_stop_words(text):
  word_list = str(text).split()
  cleaned_word_list = []
  for word in word_list:
    if word not in stop_words:
      cleaned_word_list.append(word)
  return " ".join(cleaned_word_list)

In [ ]:
def clean_emails(text):
  word_list = str(text).split()
  cleaned_word_list = []
  for word in word_list: 
    if not re.search(".*@.*",word):
      cleaned_word_list.append(word)
  return " ".join(cleaned_word_list)

In [ ]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def clean_punctuation(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
def clean_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def clean_urls(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)

In [ ]:
def clean_nums(data):
    return re.sub('[0-9]+', '', data)

In [ ]:
print("Cleaning stop words...")
df['text'] = df['text'].apply(lambda tweet: clean_stop_words(tweet))
print("Cleaning words with @...")
df['text'] = df['text'].apply(lambda tweet: clean_emails(tweet))
print("Cleaning punctuation...")
df['text'] = df['text'].apply(lambda tweet: clean_punctuation(tweet))
print("Cleaning urls...")
df['text'] = df['text'].apply(lambda tweet: clean_urls(tweet))
print("Cleaning repeating characters...")
df['text'] = df['text'].apply(lambda tweet: clean_repeating_char(tweet))
print("Cleaning numeric characters...")
df['text'] = df['text'].apply(lambda tweet: clean_nums(tweet))
print("Done cleaning")
print(df.head())

Cleaning stop words...
Cleaning words with @...
Cleaning punctuation...
Cleaning urls...
Cleaning repeating characters...
Cleaning numeric characters...
Done cleaning
   label  ...                                               text
0     -1  ...  upset cant update facebok texting it might cry...
1     -1  ...  dived many times bal managed save  rest go bounds
2     -1  ...                    whole body fels itchy like fire
3     -1  ...           no behaving al im mad here cant se there
4     -1  ...                                         whole crew

[5 rows x 6 columns]


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
df['text'] = df['text'].apply(tokenizer.tokenize)

In [ ]:
df.head()
df['text']

0          [upset, cant, update, facebok, texting, it, mi...
1          [dived, many, times, bal, managed, save, rest,...
2                     [whole, body, fels, itchy, like, fire]
3          [no, behaving, al, im, mad, here, cant, se, th...
4                                              [whole, crew]
                                 ...                        
1599994                [woke, up, schol, best, feling, ever]
1599995    [thewdbcom, col, hear, old, walt, interviews, ...
1599996                [ready, mojo, makeover, ask, details]
1599997    [hapy, th, birthday, bo, al, time, tupac, amar...
1599998                               [hapy, charitytuesday]
Name: text, Length: 1599999, dtype: object

In [ ]:
ps = PorterStemmer() 
def stemming_on_text(data):
  text = [ps.stem(word) for word in data]
  return text
  df['text'] = df['text'].apply(lambda x: stemming_on_text(x))

In [ ]:
nltk.download('wordnet')

lemmatizer = nltk.WordNetLemmatizer()
def lematzing_test(data):
  text = [lemmatizer.lemmatize(word) for word in data]
  return text

df['text'] = df['text'].apply(lambda x: lematzing_test(x))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
max_len = 500
T= Tokenizer(num_words =2000)
T.fit_on_texts(df['text'])
value = T.texts_to_sequences(df['text'])
j = sequence.pad_sequences(value,maxlen = max_len)

In [ ]:
df['label'][df['label'] == 0] = 0.5
df['label'][df['label'] == -1] = 0
df['label'][df['label'] == 1] = 1

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(j,df['label'], test_size= 0.7, random_state=42)

In [ ]:
model = Input(shape = [500]);

In [ ]:
 model = Embedding(input_dim = 2000,output_dim=500,input_length=500)(model)

In [ ]:
lstm = LSTM(64);
model = lstm(model)

In [ ]:
model = Dense(256)(model)

In [ ]:
model = Activation('relu')(model)

In [ ]:
model = Dropout(.5)(model)

In [ ]:
b = Activation('sigmoid')
model = b(model)

In [ ]:
def nm_model():
  model = Input(shape = [500]);
  layer1 = model
  model = Embedding(input_dim = 2000,output_dim=500,input_length=500)(model)
  lstm = LSTM(64);
  model = lstm(model)
  model = Dense(256)(model)
  model = Activation('relu')(model)
  model = Dropout(.5)(model)
  model = Dense(1)(model)
  b = Activation('sigmoid')
  model = b(model)
  return Model(inputs = layer1,outputs = model)

In [ ]:
model = nm_model()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy']) 
print(X_train)
history=model.fit(X_train[:40000],Y_train[:40000],batch_size=100,epochs=6, validation_split=0.1)
print('Training finished')

[[   0    0    0 ...  277  520  595]
 [   0    0    0 ...  191  193   68]
 [   0    0    0 ... 1748   54   12]
 ...
 [   0    0    0 ...   57   93  745]
 [   0    0    0 ...  346  931  146]
 [   0    0    0 ...    0    0   15]]
Epoch 1/6
360/360 [==============================] - 61s 80ms/step - loss: 0.5868 - accuracy: 0.6744 - val_loss: 0.5028 - val_accuracy: 0.7508
Epoch 2/6
360/360 [==============================] - 28s 79ms/step - loss: 0.4860 - accuracy: 0.7618 - val_loss: 0.4979 - val_accuracy: 0.7535
Epoch 3/6
360/360 [==============================] - 28s 79ms/step - loss: 0.4611 - accuracy: 0.7817 - val_loss: 0.4930 - val_accuracy: 0.7575
Epoch 4/6
360/360 [==============================] - 29s 80ms/step - loss: 0.4445 - accuracy: 0.7879 - val_loss: 0.5059 - val_accuracy: 0.7515
Epoch 5/6
360/360 [==============================] - 29s 80ms/step - loss: 0.4194 - accuracy: 0.7993 - val_loss: 0.5099 - val_accuracy: 0.7502
Epoch 6/6
360/360 [==============================] - 29s 